# 5. Encoding Strings

## Summary of Commands

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, GridSearchCV
from mymetrics import root_mean_squared_log_error

hs = pd.read_csv('data/housing_sample.csv')
X = hs[['YearBuilt', 'GrLivArea', 'GarageArea', 'LotFrontage']].values
y = hs.pop('SalePrice').values

kf = KFold(n_splits=5, shuffle=True)

si = SimpleImputer(strategy='mean')
ss = StandardScaler()
ridge = Ridge()

steps = [('impute', si), ('standardize', ss), ('ridge', ridge)]
pipe = Pipeline(steps)

grid = {'impute__strategy': ['mean', 'median'],
        'ridge__alpha': np.logspace(-5, 5)}
gs = GridSearchCV(estimator=pipe, param_grid=grid, 
                  cv=kf, scoring=root_mean_squared_log_error)
gs.fit(X, y)
pd.DataFrame(gs.cv_results_)

## String vs Numeric columns
Thus far, we haven't looked at any of the string columns. This is because string columns are not allowed to be passed into machine learning estimators in scikit-learn. They must be encoded to numeric first. Two common ways of encoding strings to numeric are one-hot and ordinal. 

### One-hot encoding
One-hot encoding is useful whenever there is no inherent order of the string values such as we see in the neighborhood column. The `OneHotEncoding` transformer is found in the `preprocessing` module.

In [ ]:
hs.head()

### Big upgrade to `OneHotEncoder` in 0.20
The `OneHotEncoder` was given a huge upgrade in version 0.20. Previously it was only able to handle numeric values.

### Using `OneHotEncoder`
It transforms each column into several new columns where the number of new columns is equal to the number of unique values in the original column. Each row will contain all 0's except for the a single column, which will be 1, to represent that string. Let's see an example of how it works with dummy data. Note that we must instantiate it with `sparse` set to `False` in order to get a visualize the array on our screens.

In [ ]:
a = np.array([['chevy'], ['ford'], ['chevy'], ['chrysler'], ['mercedes']])
a

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(a)

scikit-learn provides the method `get_feature_names` to get the name of each column.

In [ ]:
ohe.get_feature_names()

### Values not in the training set
If you wish to use this same encoding to transform another column, an error will be raised if one of the values did not appear in the training set as happens below with 'toyota'.

In [ ]:
b = np.array([['ford'], ['chrysler'], ['toyota']])
b

In [ ]:
ohe.transform(b) # error

### Handle Unknown
As a workaround, you can instantiate `OneHotEncoder` by setting `handle_unknown` to 'ignore'. The unknown value will be encoded as a row of all 0's.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe.fit_transform(a)

The last row is encoded as all 0's.

In [ ]:
ohe.transform(b)

## One-hot encode the housing dataset
Let's use the `OneHotEncoder` on both of the string columns in our dataset. To do so we will need to fill in the missing values and do so with the string 'MISSING'. We use a pipeline with the `SimpleImputer` to

In [ ]:
hs.isna().sum()

In [ ]:
X = hs[['Neighborhood', 'Exterior1st']].values
X[:5]

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

si = SimpleImputer(strategy='constant', fill_value='MISSING')
ohe = OneHotEncoder(sparse=False)
steps = [('impute', si), ('encode', ohe)]

pipe = Pipeline(steps)
X1 = pipe.fit_transform(X)
X1[:5]

In [ ]:
pipe.named_steps['encode'].get_feature_names()

## Exercise
Read in the housing_original.csv file to get the full housing dataset with many more variables. Select a subset of the string variables to encode. Use a pipeline to fill missing values if need be.

## Extra - OrdinalEncoder
A different encoding strategy is to use a single column and encode each string with a different number beginning at 0. This makes sense if the strings have a natural ordering such as descriptive feedback ratings (very bad, bad, neutral, good, very good) or the color of a diamond. If there is no natural ordering then this method would arbitrarily rank one string greater than another.

Below is a simple example with some dummy data.

In [ ]:
a = np.array([['good'], ['very good'], ['good'], ['bad'], ['very bad']], dtype='object')
a

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[['very bad', 'bad','neutral', 'good', 'very good']])
oe.fit_transform(a)